# Keras MNIST mlp

Dimension: 784x10<br>
Training size: 60,000<br>
Test size: 10,000

In [1]:
import keras as ks

Using Theano backend.


In [2]:
import numpy as np
from functools import reduce
import datetime

### Load data

In [3]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()
test_data = datahandler.load_testing()

In [4]:
train_image_array = np.asarray(train_data[0])
test_image_array = np.asarray(test_data[0])

In [5]:
train_label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    train_label_list.append(l)

train_label_array = np.asarray(train_label_list)

In [6]:
test_label_list = []
for i in test_data[1]:
    l = [0]*10
    l[i] = 1
    test_label_list.append(l)

test_label_array = np.asarray(test_label_list)

### Create and train model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD

In [8]:
model = Sequential([
    Dense(10, input_dim=784, activation = 'sigmoid'),
    Dense(10, activation = 'softmax'),
])

In [9]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [10]:
t = datetime.datetime.now()
hist = model.fit(train_image_array, 
                 train_label_array,
                 nb_epoch=10, 
                 batch_size=10)
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t))

Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.2604 - acc: 0.9035     
Epoch 2/10
60000/60000 [==============================] - 3s - loss: 0.2093 - acc: 0.9133     
Epoch 3/10
60000/60000 [==============================] - 3s - loss: 0.1725 - acc: 0.9291     
Epoch 4/10
60000/60000 [==============================] - 3s - loss: 0.1463 - acc: 0.9496     
Epoch 5/10
60000/60000 [==============================] - 3s - loss: 0.1300 - acc: 0.9565     
Epoch 6/10
60000/60000 [==============================] - 3s - loss: 0.1185 - acc: 0.9627     
Epoch 7/10
60000/60000 [==============================] - 3s - loss: 0.1112 - acc: 0.9658     
Epoch 8/10
60000/60000 [==============================] - 4s - loss: 0.1065 - acc: 0.9671     
Epoch 9/10
60000/60000 [==============================] - 4s - loss: 0.1024 - acc: 0.9684     
Epoch 10/10
60000/60000 [==============================] - 4s - loss: 0.0985 - acc: 0.9693     
Tiempo de ejecución: 0:00:40.450213


### Make predictions

In [11]:
p = model.predict([test_image_array])

In [12]:
print("Dimensión predicción: {0}".format(p.shape)) #check prediction dimensions

Dimensión predicción: (10000, 10)


# Metrics

### Confusion matrix
Useful for metric calculation

In [16]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: test_label_array[j][i], range(0,10)))[0]
    conf_mtx[actual_label][max_pred] += 1
np.set_printoptions(suppress=True)
print(conf_mtx)

[[  926.     0.    19.     3.     4.     7.    13.     2.     6.     0.]
 [    0.  1100.     6.     1.     0.     2.    15.     1.     8.     2.]
 [   10.     7.   857.    14.    23.     2.    26.    16.    73.     4.]
 [    9.     9.    47.   823.     0.    71.     7.    12.    25.     7.]
 [    1.     6.     3.     2.   864.     3.    41.     1.     3.    58.]
 [   28.     5.    14.    60.    39.   614.    28.    15.    69.    20.]
 [   20.     3.    19.     1.    44.     5.   850.     0.    16.     0.]
 [    1.    26.    29.     3.    10.     1.     3.   890.    15.    50.]
 [   18.    10.    14.    22.    22.    67.    51.     7.   752.    11.]
 [   12.    11.     2.     8.   126.    26.     7.    28.    12.   777.]]


### Accuracy

In [17]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/10000)

In [18]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.84530


### Recall

In [19]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [20]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.94490
Recall 1: 0.96916
Recall 2: 0.83043
Recall 3: 0.81485
Recall 4: 0.87984
Recall 5: 0.68834
Recall 6: 0.88727
Recall 7: 0.86576
Recall 8: 0.77207
Recall 9: 0.77007

Recall mean: 0.84227


### Precision

In [21]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [22]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.90341
Precision 1: 0.93458
Precision 2: 0.84851
Precision 3: 0.87834
Precision 4: 0.76325
Precision 5: 0.76942
Precision 6: 0.81652
Precision 7: 0.91564
Precision 8: 0.76813
Precision 9: 0.83638

Precision mean: 0.84342
